# BNP Paribas Cardif Claims Management - 初心者向けガイド

このノートブックでは、BNP Paribas Cardif Claims Managementコンペに取り組みながら、機械学習の基礎を学びます。

## 📚 このコンペについて

**問題の種類**: 二値分類（Binary Classification）
- **二値分類とは**: データを2つのカテゴリに分類する問題です
- このコンペでは、保険請求を2つのカテゴリに分類します：
  1. **承認を加速できるクレーム**（早く支払いできる）
  2. **追加情報が必要なクレーム**（承認前に確認が必要）

**評価指標**: Log Loss（対数損失）
- **Log Lossとは**: 予測の確信度も考慮した評価指標です
- 予測が確実に正しいほど良いスコアになります
- 0に近いほど良い（完璧な予測なら0）

**データ形式**: 表形式データ（Tabular Data）
- CSVファイルのような表形式のデータです
- 各行が1つの保険請求、各列が特徴（年齢、金額など）を表します


## 🎯 機械学習の基本用語

### 重要な用語集

1. **訓練データ（Training Data）**
   - モデルを学習させるためのデータ
   - 特徴（入力）と正解ラベル（出力）の両方を含む
   - 例: 過去の保険請求データと、その承認結果

2. **テストデータ（Test Data）**
   - 予測を行うためのデータ
   - 特徴のみ含まれ、正解ラベルはない
   - 例: 新しい保険請求データ（結果はまだ分からない）

3. **特徴量（Feature）**
   - 予測に使う情報（変数）
   - 例: 年齢、請求金額、契約期間など

4. **ラベル（Label / Target）**
   - 予測したい値（正解）
   - このコンペでは「承認を加速できるかどうか」（0または1）

5. **モデル（Model）**
   - データからパターンを学習するアルゴリズム
   - 訓練データを使って学習し、新しいデータを予測できるようになる

6. **前処理（Preprocessing）**
   - データをモデルが使える形に整える作業
   - 欠損値の処理、文字列を数値に変換など

7. **特徴量エンジニアリング（Feature Engineering）**
   - 新しい特徴量を作ったり、既存の特徴量を改善すること
   - 例: 年齢から「高齢者かどうか」という特徴を作る


## 📊 全体の流れ（ワークフロー）

```
1. ライブラリのインポート
   ↓
2. データの読み込み
   ↓
3. データ探索（EDA: Exploratory Data Analysis）
   - データの形、欠損値、分布を確認
   ↓
4. データ前処理
   - 欠損値の処理
   - 文字列データの数値化
   ↓
5. 特徴量エンジニアリング
   - 新しい特徴量の作成
   - 不要な特徴量の削除
   ↓
6. モデルの訓練
   - 訓練データを使ってモデルを学習
   ↓
7. 予測の生成
   - テストデータに対して予測を実行
   ↓
8. 提出ファイルの作成
   - Kaggle提出用のCSVファイルを作成
```


## ステップ1: ライブラリのインポート

**ライブラリとは**: 他の人が作った便利なプログラムの集まりです。

- **pandas**: データを扱うためのライブラリ（表形式データの読み込み・操作）
- **numpy**: 数値計算のためのライブラリ
- **matplotlib / seaborn**: グラフを描くためのライブラリ
- **sklearn**: 機械学習のためのライブラリ（モデル、前処理など）


In [ ]:
# 必要なライブラリをインポート
import pandas as pd  # データ操作
import numpy as np   # 数値計算
import matplotlib.pyplot as plt  # グラフ描画
import seaborn as sns  # より見やすいグラフ
import warnings  # 警告を非表示にする

# 機械学習関連のライブラリ
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

# 警告を非表示にする（見やすくするため）
warnings.filterwarnings('ignore')

# グラフのスタイル設定
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ ライブラリのインポートが完了しました！")


## ステップ2: データの読み込み

まず、Kaggleからデータをダウンロードする必要があります。

### データのダウンロード方法

1. **Kaggle APIを使う場合**（推奨）:
   ```bash
   kaggle competitions download -c bnp-paribas-cardif-claims-management -p input/
   cd input/
   unzip bnp-paribas-cardif-claims-management.zip
   ```

2. **KaggleのWebサイトから手動でダウンロード**:
   - コンペページの「Data」タブからダウンロード
   - `input/` フォルダに保存

### データファイルについて

通常、以下のファイルが含まれます：
- `train.csv`: 訓練データ（特徴量 + 正解ラベル）
- `test.csv`: テストデータ（特徴量のみ）
- `sample_submission.csv`: 提出ファイルの形式例


In [ ]:
# データの読み込み
# 注意: パスは実際のデータの場所に合わせて変更してください

try:
    # Kaggle環境の場合
    train_df = pd.read_csv('/kaggle/input/bnp-paribas-cardif-claims-management/train.csv')
    test_df = pd.read_csv('/kaggle/input/bnp-paribas-cardif-claims-management/test.csv')
    print("✅ Kaggle環境からデータを読み込みました")
except FileNotFoundError:
    # ローカル環境の場合
    train_df = pd.read_csv('../input/bnp-paribas-cardif-claims-management/train.csv')
    test_df = pd.read_csv('../input/bnp-paribas-cardif-claims-management/test.csv')
    print("✅ ローカル環境からデータを読み込みました")

# データの基本情報を確認
print(f"\n📊 訓練データのサイズ: {train_df.shape}")
print(f"📊 テストデータのサイズ: {test_df.shape}")
print(f"\n訓練データの最初の5行:")
train_df.head()


## ステップ3: データ探索（EDA: Exploratory Data Analysis）

**EDAとは**: データを理解するための探索的な分析です。

### 確認すべきこと

1. **データの基本情報**
   - 行数・列数
   - 各列のデータ型（数値、文字列など）
   - 欠損値の有無

2. **統計情報**
   - 数値データの平均、最大値、最小値など
   - カテゴリデータの種類と出現頻度

3. **可視化**
   - ヒストグラム（分布の確認）
   - 相関行列（特徴量同士の関係）
   - 箱ひげ図（外れ値の確認）


### オプション: 自動EDAレポート生成（ydata-profiling）

**ydata-profilingとは**: データの基本情報、統計、欠損値、相関、分布などを自動で分析してHTMLレポートを生成するライブラリです。

**使い方**:
1. インストール: `pip install ydata-profiling`
2. 1行のコードでレポート生成
3. ブラウザでHTMLレポートを確認

**注意**: データが大きい場合（10万行以上など）、時間がかかる可能性があります。
その場合は、サンプルデータで実行するか、`minimal=True`オプションを使用してください。


In [ ]:
# オプション: 自動EDAレポート生成（ydata-profiling）
# 注意: データが大きい場合は時間がかかる可能性があります

if EDA_AVAILABLE:
    print("=" * 60)
    print("📊 自動EDAレポート生成中...")
    print("=" * 60)
    print("⚠️ データが大きい場合、数分かかる可能性があります")
    print("   中断する場合は、カーネルを停止してください")
    
    # サンプルデータで実行（全データは時間がかかるため）
    # 全データで実行する場合は、以下のコメントを外してください
    sample_size = min(10000, len(train_df))  # 最大10,000行
    train_sample = train_df.sample(n=sample_size, random_state=42)
    
    print(f"\nサンプルデータ（{sample_size}行）でレポートを生成します...")
    
    # レポート生成（minimal=Trueで高速化）
    profile = ProfileReport(
        train_sample, 
        title="BNP Paribas Cardif - 訓練データ EDAレポート",
        minimal=True  # 高速化のため簡易モード
    )
    
    # HTMLレポートを保存
    profile.to_file("../eda_report_train.html")
    print("✅ レポートを生成しました: ../eda_report_train.html")
    print("   ブラウザで開いて確認してください")
    
    # ノートブック内で表示（オプション）
    # profile.to_notebook_iframe()
else:
    print("⚠️ ydata-profilingがインストールされていないため、スキップします")
    print("   インストール: pip install ydata-profiling")


In [ ]:
### 手動EDA: データの基本情報を確認

自動EDAツールを使わない場合、以下のセルで手動で確認できます。
    '欠損数': missing,
    '欠損率(%)': missing_percent
})
missing_df = missing_df[missing_df['欠損数'] > 0].sort_values('欠損数', ascending=False)
print(f"欠損値がある列: {len(missing_df)}列")
if len(missing_df) > 0:
    print(missing_df.head(20))
else:
    print("欠損値はありません！")


In [ ]:
# 目的変数（target）の分布を確認
# 注意: このコンペの目的変数名を確認する必要があります
# 一般的には 'target' や 'label' という名前です

# 目的変数の列名を探す
target_cols = [col for col in train_df.columns if col.lower() in ['target', 'label', 'y']]
if len(target_cols) == 0:
    # ID列を除いて、最後の列が目的変数の可能性が高い
    target_col = train_df.columns[-1]
    print(f"⚠️ 目的変数の列名が不明です。最後の列 '{target_col}' を目的変数として使用します。")
else:
    target_col = target_cols[0]
    print(f"✅ 目的変数の列名: '{target_col}'")

# 目的変数の分布を確認
if target_col in train_df.columns:
    print(f"\n📊 目的変数の分布:")
    print(train_df[target_col].value_counts())
    print(f"\n📊 目的変数の割合:")
    print(train_df[target_col].value_counts(normalize=True))
    
    # 可視化
    plt.figure(figsize=(8, 5))
    train_df[target_col].value_counts().plot(kind='bar')
    plt.title('目的変数の分布')
    plt.xlabel('クラス')
    plt.ylabel('件数')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 目的変数の列が見つかりませんでした。データを確認してください。")


In [ ]:
# 数値特徴量の統計情報を確認
print("=" * 60)
print("📊 数値特徴量の基本統計")
print("=" * 60)

# 数値列のみを抽出
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
# ID列と目的変数を除外
if 'ID' in numeric_cols:
    numeric_cols.remove('ID')
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

if len(numeric_cols) > 0:
    print(f"数値特徴量の数: {len(numeric_cols)}")
    print(train_df[numeric_cols].describe())
else:
    print("数値特徴量が見つかりませんでした。")


In [ ]:
# カテゴリ特徴量の確認
print("=" * 60)
print("📊 カテゴリ特徴量の確認")
print("=" * 60)

# 文字列列またはobject型の列を抽出
categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()

if len(categorical_cols) > 0:
    print(f"カテゴリ特徴量の数: {len(categorical_cols)}")
    print(f"カテゴリ特徴量の列名: {categorical_cols}")
    
    # 各カテゴリ特徴量のユニークな値の数を確認
    print("\n各カテゴリ特徴量のユニークな値の数:")
    for col in categorical_cols[:10]:  # 最初の10個だけ表示
        unique_count = train_df[col].nunique()
        print(f"  {col}: {unique_count}種類")
        if unique_count <= 10:  # 種類が少ない場合は値も表示
            print(f"    値: {train_df[col].unique()}")
else:
    print("カテゴリ特徴量が見つかりませんでした。")


## ステップ4: データ前処理

**前処理とは**: データをモデルが使える形に整える作業です。

### 主な前処理の種類

1. **欠損値の処理**
   - **欠損値とは**: データが存在しない箇所（NaN、NULLなど）
   - **処理方法**:
     - 平均値や中央値で埋める（数値データ）
     - 最頻値で埋める（カテゴリデータ）
     - 欠損値がある行を削除（データが十分ある場合）

2. **カテゴリデータの数値化**
   - **カテゴリデータとは**: 文字列や有限の値（例: "男性"、"女性"）
   - **処理方法**:
     - **Label Encoding**: 各カテゴリに数値を割り当て（例: "男性"→0, "女性"→1）
     - **One-Hot Encoding**: 各カテゴリを0/1の列に変換（例: "男性"列、"女性"列を作る）

3. **特徴量のスケーリング**
   - **スケーリングとは**: 数値の範囲を揃えること
   - 例: 年齢（0-100）と年収（0-1000万）を同じスケールに変換
   - 多くの機械学習モデルで重要（特にロジスティック回帰など）


In [ ]:
# 前処理の準備
# 訓練データとテストデータを結合して、同じ前処理を適用する
# （訓練データとテストデータで異なる処理をしないようにするため）

# ID列と目的変数を保存
train_ids = train_df['ID'].copy() if 'ID' in train_df.columns else train_df.index
test_ids = test_df['ID'].copy() if 'ID' in test_df.columns else test_df.index

# 目的変数を保存
y_train = train_df[target_col].copy() if target_col in train_df.columns else None

# 訓練データとテストデータを結合（前処理を統一するため）
# テストデータには目的変数がないので、ダミー列を追加
if target_col in train_df.columns:
    test_df_temp = test_df.copy()
    test_df_temp[target_col] = -1  # ダミー値
    combined_df = pd.concat([train_df, test_df_temp], axis=0, ignore_index=True)
else:
    combined_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

print(f"結合後のデータサイズ: {combined_df.shape}")
print(f"訓練データ: {len(train_df)}行")
print(f"テストデータ: {len(test_df)}行")


In [ ]:
# 欠損値の処理
print("=" * 60)
print("🔧 欠損値の処理")
print("=" * 60)

# 数値列の欠損値を中央値で埋める
numeric_cols_for_impute = combined_df.select_dtypes(include=[np.number]).columns.tolist()
if 'ID' in numeric_cols_for_impute:
    numeric_cols_for_impute.remove('ID')
if target_col in numeric_cols_for_impute:
    numeric_cols_for_impute.remove(target_col)

if len(numeric_cols_for_impute) > 0:
    # 欠損値がある数値列を確認
    missing_numeric = [col for col in numeric_cols_for_impute if combined_df[col].isnull().sum() > 0]
    if len(missing_numeric) > 0:
        print(f"数値列の欠損値を中央値で埋めます: {len(missing_numeric)}列")
        imputer_numeric = SimpleImputer(strategy='median')
        combined_df[numeric_cols_for_impute] = imputer_numeric.fit_transform(combined_df[numeric_cols_for_impute])
        print("✅ 数値列の欠損値処理完了")
    else:
        print("✅ 数値列に欠損値はありません")

# カテゴリ列の欠損値を最頻値で埋める
categorical_cols_for_impute = combined_df.select_dtypes(include=['object']).columns.tolist()
if len(categorical_cols_for_impute) > 0:
    missing_categorical = [col for col in categorical_cols_for_impute if combined_df[col].isnull().sum() > 0]
    if len(missing_categorical) > 0:
        print(f"カテゴリ列の欠損値を最頻値で埋めます: {len(missing_categorical)}列")
        imputer_categorical = SimpleImputer(strategy='most_frequent')
        combined_df[categorical_cols_for_impute] = imputer_categorical.fit_transform(combined_df[categorical_cols_for_impute])
        print("✅ カテゴリ列の欠損値処理完了")
    else:
        print("✅ カテゴリ列に欠損値はありません")


In [ ]:
# カテゴリデータの数値化（Label Encoding）
print("=" * 60)
print("🔧 カテゴリデータの数値化")
print("=" * 60)

label_encoders = {}  # 後で使うために保存

categorical_cols_to_encode = combined_df.select_dtypes(include=['object']).columns.tolist()

if len(categorical_cols_to_encode) > 0:
    print(f"カテゴリ列を数値化します: {len(categorical_cols_to_encode)}列")
    for col in categorical_cols_to_encode:
        le = LabelEncoder()
        # 訓練データとテストデータを結合した状態でfit（一貫性を保つため）
        combined_df[col] = le.fit_transform(combined_df[col].astype(str))
        label_encoders[col] = le
    print("✅ カテゴリデータの数値化完了")
else:
    print("✅ カテゴリデータはありません")


In [ ]:
# 訓練データとテストデータを再度分離
train_processed = combined_df[:len(train_df)].copy()
test_processed = combined_df[len(train_df):].copy()

# 目的変数を復元
if y_train is not None:
    train_processed[target_col] = y_train

# ID列と目的変数を除外して特徴量のみにする
exclude_cols = ['ID']
if target_col in train_processed.columns:
    exclude_cols.append(target_col)

feature_cols = [col for col in train_processed.columns if col not in exclude_cols]

X_train = train_processed[feature_cols].copy()
y_train_final = train_processed[target_col].copy() if target_col in train_processed.columns else None
X_test = test_processed[feature_cols].copy()

print(f"✅ 前処理完了")
print(f"訓練データの特徴量数: {X_train.shape[1]}")
print(f"訓練データのサンプル数: {X_train.shape[0]}")
print(f"テストデータのサンプル数: {X_test.shape[0]}")


## ステップ5: モデルの訓練

**モデルとは**: データからパターンを学習して予測を行うアルゴリズムです。

### このコンペで使うモデル

1. **ロジスティック回帰（Logistic Regression）**
   - シンプルで解釈しやすいモデル
   - 線形関係を学習する
   - ベースラインとしてよく使われる

2. **ランダムフォレスト（Random Forest）**
   - 複数の決定木を組み合わせたモデル
   - 非線形関係も学習できる
   - 特徴量の重要度が分かる

3. **勾配ブースティング（Gradient Boosting）**
   - 弱い学習器を順番に組み合わせて強くする
   - 高い精度が出やすい
   - XGBoost、LightGBMなどが有名

### 交差検証（Cross Validation）

**交差検証とは**: データを複数のグループに分けて、それぞれで訓練・評価を繰り返す方法です。
- モデルの性能をより正確に評価できる
- 過学習（訓練データにだけ適合してしまうこと）を防ぐ


In [ ]:
# モデル1: ロジスティック回帰（ベースライン）
print("=" * 60)
print("🤖 モデル1: ロジスティック回帰")
print("=" * 60)

if y_train_final is not None:
    # データのスケーリング（ロジスティック回帰には重要）
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # モデルの訓練
    lr_model = LogisticRegression(random_state=42, max_iter=1000)
    lr_model.fit(X_train_scaled, y_train_final)
    
    # 訓練データでの予測（評価用）
    y_train_pred_lr = lr_model.predict_proba(X_train_scaled)[:, 1]
    train_logloss_lr = log_loss(y_train_final, y_train_pred_lr)
    
    print(f"訓練データでのLog Loss: {train_logloss_lr:.6f}")
    
    # テストデータでの予測
    y_test_pred_lr = lr_model.predict_proba(X_test_scaled)[:, 1]
    
    print("✅ ロジスティック回帰の訓練完了")
else:
    print("⚠️ 目的変数が見つかりませんでした")


In [ ]:
# モデル2: ランダムフォレスト
print("=" * 60)
print("🤖 モデル2: ランダムフォレスト")
print("=" * 60)

if y_train_final is not None:
    # ランダムフォレストはスケーリング不要
    rf_model = RandomForestClassifier(
        n_estimators=100,  # 決定木の数
        max_depth=10,      # 木の深さ
        random_state=42,
        n_jobs=-1          # 並列処理
    )
    
    rf_model.fit(X_train, y_train_final)
    
    # 訓練データでの予測
    y_train_pred_rf = rf_model.predict_proba(X_train)[:, 1]
    train_logloss_rf = log_loss(y_train_final, y_train_pred_rf)
    
    print(f"訓練データでのLog Loss: {train_logloss_rf:.6f}")
    
    # 特徴量の重要度を表示（上位10個）
    feature_importance = pd.DataFrame({
        '特徴量': X_train.columns,
        '重要度': rf_model.feature_importances_
    }).sort_values('重要度', ascending=False)
    
    print("\n📊 特徴量の重要度（上位10個）:")
    print(feature_importance.head(10))
    
    # テストデータでの予測
    y_test_pred_rf = rf_model.predict_proba(X_test)[:, 1]
    
    print("\n✅ ランダムフォレストの訓練完了")
else:
    print("⚠️ 目的変数が見つかりませんでした")


In [ ]:
# モデル3: 勾配ブースティング
print("=" * 60)
print("🤖 モデル3: 勾配ブースティング")
print("=" * 60)

if y_train_final is not None:
    gb_model = GradientBoostingClassifier(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    )
    
    gb_model.fit(X_train, y_train_final)
    
    # 訓練データでの予測
    y_train_pred_gb = gb_model.predict_proba(X_train)[:, 1]
    train_logloss_gb = log_loss(y_train_final, y_train_pred_gb)
    
    print(f"訓練データでのLog Loss: {train_logloss_gb:.6f}")
    
    # テストデータでの予測
    y_test_pred_gb = gb_model.predict_proba(X_test)[:, 1]
    
    print("✅ 勾配ブースティングの訓練完了")
    
    # モデルの比較（変数が定義されている場合のみ）
    print("\n" + "=" * 60)
    print("📊 モデル比較（訓練データでのLog Loss）")
    print("=" * 60)
    try:
        print(f"ロジスティック回帰: {train_logloss_lr:.6f}")
    except NameError:
        print("ロジスティック回帰: 未実行")
    try:
        print(f"ランダムフォレスト: {train_logloss_rf:.6f}")
    except NameError:
        print("ランダムフォレスト: 未実行")
    print(f"勾配ブースティング: {train_logloss_gb:.6f}")
else:
    print("⚠️ 目的変数が見つかりませんでした")


## ステップ6: 予測と提出ファイルの作成

**提出ファイルとは**: Kaggleに提出するためのCSVファイルです。
- 必須の列: `ID`（テストデータのID）と `PredictedProb`（予測確率）
- 形式は `sample_submission.csv` を参考にします

### 予測確率について

- **予測確率（PredictedProb）**: 0から1の間の値
  - 0に近い = クラス0（例: 追加情報が必要）の可能性が高い
  - 1に近い = クラス1（例: 承認を加速できる）の可能性が高い
  - 0.5 = どちらか分からない

### Log Lossの制約

Kaggleの提出では、予測確率が0や1にならないように制限されます：
- 最小値: 10^-15
- 最大値: 1 - 10^-15

これは、Log Lossの計算で0や1が入ると無限大になってしまうためです。


In [ ]:
# 提出ファイルの作成
# 最も良い性能を示したモデルを使う（この例では勾配ブースティング）

print("=" * 60)
print("📝 提出ファイルの作成")
print("=" * 60)

# 予測確率を取得（最も良いモデルを選択）
# ここでは勾配ブースティングを使いますが、実際には最も良いモデルを選んでください
try:
    best_predictions = y_test_pred_gb.copy()
    print("✅ 勾配ブースティングの予測を使用")
except NameError:
    try:
        best_predictions = y_test_pred_rf.copy()
        print("✅ ランダムフォレストの予測を使用")
    except NameError:
        try:
            best_predictions = y_test_pred_lr.copy()
            print("✅ ロジスティック回帰の予測を使用")
        except NameError:
            print("⚠️ エラー: 予測結果が見つかりませんでした")
            best_predictions = None

if best_predictions is not None:
    # Log Lossの制約を適用（0や1にならないようにする）
    epsilon = 1e-15
    best_predictions = np.clip(best_predictions, epsilon, 1 - epsilon)
    
    # 提出ファイルを作成
    submission = pd.DataFrame({
        'ID': test_ids,
        'PredictedProb': best_predictions
    })
    
    # ファイルを保存
    submission_file = '../submission_bnp_paribas.csv'
    submission.to_csv(submission_file, index=False)
    
    print(f"✅ 提出ファイルを作成しました: {submission_file}")
    print(f"\n提出ファイルの最初の10行:")
    print(submission.head(10))
    print(f"\n提出ファイルの統計情報:")
    print(submission['PredictedProb'].describe())
else:
    print("⚠️ 提出ファイルを作成できませんでした")


## 📚 次のステップ（改善のヒント）

### 1. 特徴量エンジニアリング
- 既存の特徴量から新しい特徴量を作る
- 例: 2つの特徴量を掛け算、割り算する
- 例: カテゴリ特徴量の組み合わせ

### 2. モデルのチューニング
- **ハイパーパラメータ**: モデルの設定値（例: 決定木の数、深さ）
- グリッドサーチやランダムサーチで最適な値を探す

### 3. アンサンブル
- 複数のモデルの予測を組み合わせる
- 例: 3つのモデルの予測の平均を取る

### 4. より高度なモデル
- XGBoost、LightGBM、CatBoostなどの勾配ブースティングライブラリ
- ニューラルネットワーク

### 5. 交差検証の活用
- 訓練データを分割して、より正確な性能評価を行う
- 過学習を防ぐ

## 🎉 お疲れ様でした！

このノートブックで基本的な機械学習の流れを学びました。
次は、上記の改善方法を試して、スコアを向上させてみましょう！
